In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai import *
from fastai.vision import *

In [ ]:
classes=['cricket','baseball']

In [ ]:
folder='baseball'
file='baseball.txt'

In [ ]:
path = Path('data/sports')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [ ]:
!cp ../input/* {path}/

In [ ]:
download_images(path/file, dest, max_pics=200)

In [ ]:
path.ls()

In [ ]:
folder='cricket'
file='cricket.txt'

In [ ]:
path=Path('data/sports')
dest=path/folder
dest.mkdir(parents=True,exist_ok=True)

In [ ]:
download_images(path/file,dest,max_pics=200)

In [ ]:
path.ls()

In [ ]:
for c in classes:
    print(c)
    verify_images(path/c,delete=True,max_size=500)
    

In [ ]:
np.random.seed(42)
data=ImageDataBunch.from_folder(path,train='.',valid_pct=0.2,ds_tfms=get_transforms(),size=64,num_workers=0).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(3,4))

In [ ]:
learn=cnn_learner(data,models.resnet34,metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save("stage-1")

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5,max_lr=slice(1e-03,1e-02))

In [ ]:
learn.save("stage-2")

In [ ]:
learn.load("stage-2")

In [ ]:
interp=ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(12,12))

In [ ]:
interp.plot_confusion_matrix()


In [ ]:
from fastai.widgets import *

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn, n_imgs=100)
ImageCleaner(ds, idxs, path)

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn)

In [ ]:
import fastai
fastai.device = torch.device('cpu')


In [ ]:
img = open_image(path/'cricket'/'00000021.jpg')
img

In [ ]:
classes=['baseball','cricket']

In [ ]:
data2=ImageDataBunch.single_from_classes(path,classes,ds_tfms=get_transforms(),size=224).normalize(imagenet_stats)

In [ ]:
learn=cnn_learner(data2,models.resnet34).load("stage-2")

In [ ]:
pred_class,pred_idx,outputs = learn.predict(img)
pred_class